In [1]:
import os
import pandas as pd
from pathlib import Path
from pprint import pprint
from typing import List, Dict
from llama_index.core.schema import NodeWithScore
from sentence_transformers import SentenceTransformer, util
import torch
import hashlib
print("📂 Current working directory:", Path().resolve())
print("📄 Contents:", os.listdir())


📂 Current working directory: /Users/ywxiu/jasp-multimodal-rag/notebooks
📄 Contents: ['03_video_query.ipynb', '01_pdf_retrieval.ipynb', '02_video_loader.ipynb', '__pycache__', '01_pdf_loader_llamaindex.ipynb', '01_pdf_loader_page-level-summarization.ipynb', 'data']


In [2]:
import sys
from pathlib import Path

# add /Users/ywxiu/jasp-multimodal-rag/src to sys.path
project_root = Path.cwd().parent  # goes from notebooks → jasp-multimodal-rag
src_path = project_root / "src"
sys.path.append(str(src_path))

In [3]:

from retrieval.hybrid_retrieval import (
    build_vector_retriever,
    build_bm25_retriever,
    load_docs_for_bm25,
    rrf_fuse,
    rerank_cross_encoder,
    CHROMA_DIR,
    COLLECTION_NAME,
    EMBED_MODEL,
    CHUNKS_JSON,
    K_SEMANTIC,
    K_BM25,
    RRF_K,
    TOP_AFTER_RRF,
    TOP_FINAL,
)

print("✅ hybrid_retrieval imported successfully!")


✅ hybrid_retrieval imported successfully!


In [4]:

# 1️⃣ Choose your query
query = "How to split data files in JASP?"
CHROMA_DIR = "/Users/ywxiu/jasp-multimodal-rag/data/vector_store/chroma_db"


semantic_retriever = build_vector_retriever(
    persist_dir=CHROMA_DIR,
    collection_name=COLLECTION_NAME,
    embed_model_name=EMBED_MODEL,
)
sem_results = semantic_retriever.retrieve(query)

print(f"\nSemantic top-{K_SEMANTIC}: {len(sem_results)}\n{'='*60}")

for i, item in enumerate(sem_results, 1):
    node = item.node
    metadata = node.metadata
    text = node.text.strip().replace('\n', ' ')
    print(f"\n🔹 Result {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🎯 Score  : {item.score:.3f}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Semantic top-10: 10

🔹 Result 1
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🎯 Score  : 0.510
   🧩 Chunk  : 1_0_0
   🗒️ Text   : 9    JASP  SPLITTING DATA FILES If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group to the Split box.  Variables             Descriptive Statis...
------------------------------------------------------------

🔹 Result 2
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🎯 Score  : 0.506
   🧩 Chunk  : 1_1_0
   🗒️ Text   : Deviation             6.424     7.040                                     Skewness                   0.200    -0.176                                     Std. Error of Skewness     0.137     0.110                                     Kurtosis            -0.101          -0.397 Split                    ...
------------------------------------------------------------

🔹 Result 3
   

In [5]:




docs_for_bm25 = load_docs_for_bm25(CHUNKS_JSON)
bm25_retriever = build_bm25_retriever(docs_for_bm25)
kw_results = bm25_retriever.retrieve(query)

print(f"BM25 top-{K_BM25}: {len(kw_results)}")


2025-10-30 14:46:44.696 | INFO     | retrieval.hybrid_retrieval:load_docs_for_bm25:120 - BM25 corpus: 12 documents
2025-10-30 14:46:44.697 | INFO     | retrieval.hybrid_retrieval:build_bm25_retriever:168 - Initializing BM25 retriever with 12 documents...


BM25 top-10: 10


In [6]:
print(f"\nBM25 top-{K_BM25}: {len(kw_results)}\n{'='*60}")

for i, item in enumerate(kw_results, 1):
    # BM25 retrievers often return Document-like objects
    metadata = getattr(item, "metadata", {})
    text = getattr(item, "text", str(item))  # fallback if structure differs

    print(f"\n🔹 Result {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)



BM25 top-10: 10

🔹 Result 1
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🧩 Chunk  : 1_0_0
   🗒️ Text   : 9    JASP

SPLITTING DATA FILES
If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced
for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group
to the Split box.

Variables             Descriptive Statis...
------------------------------------------------------------

🔹 Result 2
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🧩 Chunk  : 1_1_0
   🗒️ Text   : Deviation             6.424     7.040
                                    Skewness                   0.200    -0.176
                                    Std. Error of Skewness     0.137     0.110
                                    Kurtosis            -0.101          -0.397
Split                    ...
------------------------------------------------------------

🔹 Result 3
   📄 Source : test_pages25-28.pdf
   📑 Page   :

In [7]:
fused = rrf_fuse(sem_results, kw_results, k=RRF_K, top_n=TOP_AFTER_RRF)
print(f"\n🔀 RRF shortlist: {len(fused)}\n{'='*60}")

for i, item in enumerate(fused, 1):
    node = getattr(item, "node", item)
    metadata = getattr(node, "metadata", {})
    text = getattr(node, "text", str(node))

    print(f"\n🔹 Fused Result {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🎯 Score  : {getattr(item, 'score', 'N/A')}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)



🔀 RRF shortlist: 10

🔹 Fused Result 1
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🎯 Score  : 0.509631978583911
   🧩 Chunk  : 1_0_0
   🗒️ Text   : 9    JASP

SPLITTING DATA FILES
If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced
for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group
to the Split box.

Variables             Descriptive Statis...
------------------------------------------------------------

🔹 Fused Result 2
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🎯 Score  : 0.5059930258028726
   🧩 Chunk  : 1_1_0
   🗒️ Text   : Deviation             6.424     7.040
                                    Skewness                   0.200    -0.176
                                    Std. Error of Skewness     0.137     0.110
                                    Kurtosis            -0.101          -0.397
Split                    ...
---------------------------------------

In [8]:
reranked = rerank_cross_encoder(fused, query, top_k=TOP_FINAL)
print(f"\n🏁 Reranked top-{TOP_FINAL}: {len(reranked)}\n{'='*60}")

for i, item in enumerate(reranked, 1):
    node = getattr(item, "node", item)
    metadata = getattr(node, "metadata", {})
    text = getattr(node, "text", str(node))

    print(f"\n⭐ Final Rank {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🔢 New Score : {getattr(item, 'score', 'N/A')}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)

2025-10-30 14:46:44.735 | INFO     | retrieval.hybrid_retrieval:rerank_cross_encoder:221 - Cross-encoder rerank with SentenceTransformer: cross-encoder/ms-marco-MiniLM-L-6-v2



🏁 Reranked top-3: 3

⭐ Final Rank 1
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🔢 New Score : 5.365289211273193
   🧩 Chunk  : 1_0_0
   🗒️ Text   : 9    JASP

SPLITTING DATA FILES
If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced
for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group
to the Split box.

Variables             Descriptive Statis...
------------------------------------------------------------

⭐ Final Rank 2
   📄 Source : test_pages25-28.pdf
   📑 Page   : 2
   🔢 New Score : 2.429915428161621
   🧩 Chunk  : 2_4_1
   🗒️ Text   : (4) The image presents a scatterplot matrix derived from the JASP statistical software. Each point in the plot represents an individual data entry, and each cell within the grid is filled with a circle representing one of these entries. The size of the circles varies based on their position along th...
--------------------------------------

In [9]:


# 3️⃣ Helper to format text
def short(txt, n=300):
    t = txt.strip().replace("\n", " ")
    return t[:n] + ("..." if len(t) > n else "")

def extract_info(nodes):
    data = []
    for i, n in enumerate(nodes, start=1):
        md = n.node.metadata or {}
        src = md.get("source") or md.get("file") or md.get("doc_id") or "unknown"
        page = md.get("page")
        tag = f"{src}" + (f":p{page}" if page is not None else "")
        data.append({
            "rank": i,
            "source": tag,
            "score": getattr(n, "score", None),
            "text": short(n.node.get_content()),
        })
    return data

# 4️⃣ Convert all results to DataFrames
df_sem = pd.DataFrame(extract_info(sem_results))
df_bm25 = pd.DataFrame(extract_info(kw_results))
df_rrf = pd.DataFrame(extract_info(fused))
df_rerank = pd.DataFrame(extract_info(reranked))

# 5️⃣ Combine top results side by side
max_rows = max(len(df_sem), len(df_bm25), len(df_rerank))
df_combined = pd.DataFrame({
    "Semantic (BM25 embedding)": df_sem["text"].reindex(range(max_rows)),
    "BM25 (lexical)": df_bm25["text"].reindex(range(max_rows)),
    "RRF fused": df_rrf["text"].reindex(range(max_rows)),
    "Final reranked": df_rerank["text"].reindex(range(max_rows)),
})
display(df_combined.fillna(""))

# 6️⃣ Optional: save for later manual review
# df_combined.to_csv("../notebooks/retrieval_debug_output.csv", index=False)


,Semantic (BM25 embedding),BM25 (lexical),RRF fused,Final reranked
0,9 JASP SPLITTING DATA FILES If there is a ...,9 JASP SPLITTING DATA FILES If there is a ...,9 JASP SPLITTING DATA FILES If there is a ...,9 JASP SPLITTING DATA FILES If there is a ...
1,Deviation 6.424 7.040 ...,Deviation 6.424 7.040 ...,Deviation 6.424 7.040 ...,(4) The image presents a scatterplot matrix de...
2,"The variables include the mean value, standard...",9 JASP The Distribution plot is based on s...,9 JASP The Distribution plot is based on s...,Adding to this effect are two diagonal lines e...
3,(4) The image presents a scatterplot matrix de...,(4) The image presents a scatterplot matrix de...,(4) The image presents a scatterplot matrix de...,
4,9 JASP The Distribution plot is based on s...,9 JASP Q-Q Plot Variable 4 20 2 0 ...,"The variables include the mean value, standard...",
5,Adding to this effect are two diagonal lines e...,Adding to this effect are two diagonal lines e...,Adding to this effect are two diagonal lines e...,
6,9 JASP CUSTOMISABLE PLOTS ...,9 JASP CUSTOMISABLE PLOTS ...,9 JASP Q-Q Plot Variable 4 20 2 0 ...,
7,9 JASP Q-Q Plot Variable 4 20 2 0 ...,The boxplots visualise several statistics desc...,9 JASP CUSTOMISABLE PLOTS ...,
8,"It features three bars of varying heights, sym...","The variables include the mean value, standard...","It features three bars of varying heights, sym...",
9,All bars have horizontal lines extending from ...,"It features three bars of varying heights, sym...",The boxplots visualise several statistics desc...,


In [16]:
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.core.schema import NodeWithScore
from typing import List
model= "mistral:7b"

def generate_final_answer(query: str, top_nodes: List[NodeWithScore]) -> str:
    """
    Generate a context-grounded answer using Llama 3.1 from Ollama.
    Uses reranked top documents as context.
    """
    # 1️⃣ Initialize local Llama model from Ollama
    llm = Ollama(model=model, request_timeout=600)

    # 2️⃣ Build context string
    context = "\n\n".join(
        [f"Source {i+1}:\n{n.node.get_content()}" for i, n in enumerate(top_nodes)]
    )

    # 3️⃣ Define a structured prompt
    prompt_template = PromptTemplate(
        "You are a helpful assistant. Answer the question below based on the provided sources.\n\n"
        "Question:\n{query}\n\n"
        "Sources:\n{context}\n\n"
        "Please provide a concise, factual answer, followed by a short summary of which sources were most relevant."
    )

    prompt = prompt_template.format(query=query, context=context)

    # 4️⃣ Generate final response
    response = llm.complete(prompt)
    return response.text


In [17]:
# Assume you already have: query, top3_docs = rerank_cross_encoder(...)

final_answer = generate_final_answer(query, reranked)

print("🧠 Final Answer:")
print(final_answer)


KeyboardInterrupt: 

In [18]:
print(type(reranked))
print(len(reranked))
print(type(reranked[0]))
print(reranked[0].node.get_content()[:200])


<class 'list'>
3
<class 'llama_index.core.schema.NodeWithScore'>
9    JASP

SPLITTING DATA FILES
If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced
for each group. Using Descriptive data.csv with the variable 


In [19]:
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
import sys
from typing import List
from llama_index.core.schema import NodeWithScore

def generate_final_answer(query: str, top_nodes: List[NodeWithScore]) -> str:
    """Generate concise answer using a local model with streaming and safe truncation."""

    # 1️⃣ Use a smaller local model
    llm = Ollama(model="mistral:7b", request_timeout=90)

    # 2️⃣ Build and truncate context safely
    context = ""
    for i, n in enumerate(top_nodes):
        text = n.node.get_content() if hasattr(n, "node") else str(n)
        # truncate each to 600 chars for safety
        context += f"\n\n[Document {i+1}]\n{text[:600]}"
    print(f"🧩 Context length: {len(context)} characters")

    if len(context) > 6000:
        context = context[:6000]  # hard cap

    # 3️⃣ Prompt template
    prompt = f"""
You are a helpful assistant. Use the following context to answer the question.
Be concise, factual, and mention which document(s) you used.

Question:
{query}

Context:
{context}

Answer:
"""
    print("🚀 Generating answer...\n")

    # 4️⃣ Stream the response to console
    stream = llm.stream_complete(prompt)
    response_text = ""
    for token in stream:
        sys.stdout.write(token.delta)
        sys.stdout.flush()
        response_text += token.delta
    print("\n✅ Generation done.")
    return response_text


In [20]:
final_answer = generate_final_answer(query, reranked)
print(list(final_answer))

🧩 Context length: 1845 characters
🚀 Generating answer...

 To split data files in JASP, follow these steps as mentioned in Document 1:

1. Ensure you have a grouping variable (categorical or ordinal) in your data file.
2. Load the data file into JASP and add the grouping variable to the Split box.
3. Select Descriptive statistics for the variables you are interested in.
4. The software will then produce descriptive statistics, such as mean and standard error of mean, separately for each group defined by the split variable.
✅ Generation done.
[' ', 'T', 'o', ' ', 's', 'p', 'l', 'i', 't', ' ', 'd', 'a', 't', 'a', ' ', 'f', 'i', 'l', 'e', 's', ' ', 'i', 'n', ' ', 'J', 'A', 'S', 'P', ',', ' ', 'f', 'o', 'l', 'l', 'o', 'w', ' ', 't', 'h', 'e', 's', 'e', ' ', 's', 't', 'e', 'p', 's', ' ', 'a', 's', ' ', 'm', 'e', 'n', 't', 'i', 'o', 'n', 'e', 'd', ' ', 'i', 'n', ' ', 'D', 'o', 'c', 'u', 'm', 'e', 'n', 't', ' ', '1', ':', '\n', '\n', '1', '.', ' ', 'E', 'n', 's', 'u', 'r', 'e', ' ', 'y', 'o',

In [22]:
from ollama import Client
import sys
from typing import List
from llama_index.core.schema import NodeWithScore

def generate_final_answer(
    query: str,
    top_nodes: List[NodeWithScore],
    model: str = "mistral:7b",
    max_chars_per_doc: int = 800,
) -> str:
    """
    Generate an answer with direct Ollama, including detailed references (source link/chapter/page).
    """

    client = Client()

    # ---------- Build contextual snippets with reference metadata ----------
    context_blocks = []
    structured_refs = []  # keep for summary display

    for i, node in enumerate(top_nodes, 1):
        meta = getattr(node.node, "metadata", {})
        source = meta.get("source", "N/A")
        page = meta.get("page", "N/A")
        chapter = meta.get("chapter", "N/A")
        link = meta.get("link", None)
        chunk_id = meta.get("chunk_id", i)
        text = node.node.get_content()[:max_chars_per_doc]

        # Format context block for model
        context_blocks.append(
            f"[Document {i}] "
            f"(Source: {source}, Page: {page}, Chapter: {chapter}, Chunk: {chunk_id}"
            + (f", Link: {link}" if link else "") +
            f")\n{text}"
        )

        # Save structured reference summary
        structured_refs.append({
            "doc_id": i,
            "source": source,
            "page": page,
            "chapter": chapter,
            "link": link,
            "chunk_id": chunk_id
        })

    context = "\n\n".join(context_blocks)
    print(f"🧩 Total context length: {len(context)} characters\n")

    # ---------- Prompt design ----------
    prompt = f"""
You are a documentation assistant that helps users find accurate answers from official sources.

Use the provided context to answer the question.
Whenever you include a fact, cite the corresponding source by its document number,
and show key metadata (page, chapter, or link) for traceability.

Question:
{query}

Context:
{context}

Instructions:
- Answer concisely and accurately based on the context.
- Include references in the format:
  [Doc 1 – Source: test_pages25-28.pdf, Page: 3, Chapter: 'Descriptive Stats', Link: example.com]
- Summarize which documents were most relevant.
- If multiple docs mention the same concept, combine them logically.

Answer:
"""

    # ---------- Stream generation ----------
    print("🚀 Generating response...\n")
    stream = client.generate(model=model, prompt=prompt, stream=True)

    full_output = ""
    for chunk in stream:
        token = chunk.get("response", "")
        sys.stdout.write(token)
        sys.stdout.flush()
        full_output += token

    print("\n✅ Generation completed.")
    print("\n📚 References used in context:")
    for ref in structured_refs:
        print(f" - Doc {ref['doc_id']}: {ref['source']} (Page {ref['page']}, Chapter {ref['chapter']}, Link: {ref['link']})")

    return full_output


In [23]:
final_answer = generate_final_answer(query, reranked)
print(list(final_answer))

🧩 Total context length: 2500 characters

🚀 Generating response...

 To split data files in JASP, you can use the "Split" box in the interface. Add a grouping variable to the Split box, then select the desired variable from the Variables box ([Doc 1]). This will allow you to generate descriptive statistics and plots for each group separately.

For visual representation of the data, JASP offers scatterplot matrices that provide a circular point for each individual data entry ([Doc 2 & Doc 3]). The size of the circles may vary based on their position along the x and y axes, representing variable importance or correlation.

Relevant documents:
- [Doc 1 – Source: test_pages25-28.pdf, Page: 1, Chapter: 'Descriptive Stats']
- [Doc 2 – Source: test_pages25-28.pdf, Page: 2, Chapter: N/A]
- [Doc 3 – Source: test_pages25-28.pdf, Page: 2, Chapter: N/A]
✅ Generation completed.

📚 References used in context:
 - Doc 1: test_pages25-28.pdf (Page 1, Chapter N/A, Link: None)
 - Doc 2: test_pages25-28.pd